In [1]:
import random
import duckdb
import pandas as pd
random.seed(42)

Participants aux réunions:

In [3]:
person_names = ["Benjamin", "Florian", "Tarik", "Bob", "Sirine", "Alice"]

Création de la donnée sur les meetings:

In [4]:
meetings_data = []
for meeting_id in range(150):
    persons_in_meet = random.sample(person_names, random.randint(1,5))
    for person_name in persons_in_meet:
        meetings_data.append((meeting_id, person_name))


meetings_df = pd.DataFrame(meetings_data, columns=["meeting_id", "person_name"])

Durées des meetings:

In [5]:
meeting_durations = []
for meeting_id in meetings_df["meeting_id"].unique():
    duration = random.randint(10, 60)  # You can adjust the range as needed
    meeting_durations.append((meeting_id, duration))

durations_df = pd.DataFrame(meeting_durations, columns=["meeting_id", "duration_minutes"])

Tweak de la donnée pour rendre l'exercice plus fun ;)

In [6]:
average_duration = durations_df["duration_minutes"].mean()
meetings_with_flo = meetings_df[meetings_df["person_name"] == "Florian"]["meeting_id"].unique()
# meetings_with_ben = meetings_df[meetings_df["person_name"] == "Benjamin"]["meeting_id"].unique()
# s=set(meetings_with_ben) & set(meetings_with_flo)
for _, row in durations_df.iterrows():
    if row["meeting_id"] in meetings_with_flo:
        row["duration_minutes"] += random.randint(50, 55)  # Add extra minutes to meet the condition

Total:

In [7]:
merged_df = meetings_df.merge(durations_df, on="meeting_id")
merged_df

,meeting_id,person_name,duration_minutes
0,0,Benjamin,36
1,1,Florian,84
2,1,Alice,84
3,1,Sirine,84
4,2,Alice,44
...,...,...,...
430,147,Florian,83
431,147,Bob,83
432,148,Alice,63
433,148,Florian,63


<blockquote> Mais pourquoi on n'a pas mis tous les participants dans des colonnes différentes pour avoir une seule ligne par meeting ?!</blockquote> 

# Exercice: self join

Pour calculer le temps que je passe en réunion en fonction de qui d'autre est présent, il faut:
- créer une table avec toutes les combinaisons de personnes ayant assisté au même meeting
- ne garder que les records qui me concernent (Benjamin)
- enlever les records où je suis en réunion "avec moi-même"

In [22]:
%load solutions/3self_join_meetings_python.py

,meeting_id,person_name_L,person_name_R,duration_minutes
12,3,Benjamin,Sirine,39
13,3,Benjamin,Bob,39
14,3,Benjamin,Alice,39
15,3,Benjamin,Tarik,39
58,8,Benjamin,Florian,87
...,...,...,...,...
1512,144,Benjamin,Florian,66
1513,144,Benjamin,Tarik,66
1514,144,Benjamin,Sirine,66
1528,146,Benjamin,Sirine,105


In [24]:
%load solutions/4self_join_meetings_sql.py

┌────────────┬─────────────┬──────────────────┬─────────────┬──────────────────┐
│ meeting_id │ person_name │ duration_minutes │ person_name │ duration_minutes │
│   int64    │   varchar   │      int64       │   varchar   │      int64       │
├────────────┼─────────────┼──────────────────┼─────────────┼──────────────────┤
│          3 │ Benjamin    │               39 │ Tarik       │               39 │
│          8 │ Benjamin    │               87 │ Tarik       │               87 │
│         11 │ Benjamin    │               54 │ Bob         │               54 │
│         13 │ Benjamin    │               58 │ Tarik       │               58 │
│         15 │ Benjamin    │               34 │ Bob         │               34 │
│         17 │ Benjamin    │               10 │ Tarik       │               10 │
│         18 │ Benjamin    │               32 │ Bob         │               32 │
│         25 │ Benjamin    │              110 │ Alice       │              110 │
│         26 │ Benjamin    │

<blockquote> Super ton exercice, mais ça nous aide pas à répondre à la question. Comment on fait avec cette table pour savoir avec qui on passe le plus de temps ?! </blockquote>

Pour répondre à cette question, il va falloir faire la moyenne de durée de réunion _par_ collègue.

C'est une agrégation, et maintenant qu'on a fait le tour des joins, on va pouvoir attaquer cette partie :) 